In [1]:
from simple_salesforce import Salesforce, SalesforceAuthenticationFailed, SalesforceMalformedRequest
import pandas as pd
try:
# Autenticación
    sf = Salesforce(
        username='yonatan.osorio@openenglish.com',
        password='Yonatan26++',
        security_token='ho7zyIcpDbybcP1G7tLbor6sU'
    )

    print("Conexion exitosa: "+sf.sf_instance)
    print("Versión: "+sf.sf_version)

    # Consulta SOQL (puedes ajustarla a tu necesidad)
    soql_query = """
    SELECT 
    AccountId,
    Account.CaseSafe_Id__c,
    CaseNumber,
    Account.Alt_Email__c,
    Description,
    Status,
    Account.CreatedDate,
    Closed_Date__c,
    Case_Owner_read_only__c,
    LeadName__c,
    Organization__c,
    Sub_Status_Original__c,
    Sub_Status_Recovery__c,
    Sub_Status__c,
    Account.BillingCountry,
    LeadCountry__c,
    Type,
    Subject,
    Related_Country__c 
    FROM Case 
    WHERE Account.BillingCountry!='TR' 
    AND Account.BillingCountry!='BR'
    AND Type IN ('Sales Recovery', 'Sales Recovery - Upsell Renewal')
    AND Account.CreatedDate = THIS_MONTH
    AND Account.CreatedDate != TODAY
    LIMIT 100000
    """

    # Ejecutar la consulta
    resultado = sf.query_all(soql_query)
    registros = resultado['records']

    # Normalizar datos relacionados (como Account)
    for r in registros:
        r.pop('attributes', None)
        account = r.get('Account')
        if account:
            r['Account.CaseSafe_Id__c'] = account.get('CaseSafe_Id__c')
            r['Account.BillingCountry'] = account.get('BillingCountry')
            r['Account.CreatedDate'] = account.get('CreatedDate')
        else:
            r['Account.CaseSafe_Id__c'] = None
            r['Account.BillingCountry'] = None
            r['Account.CreatedDate'] = None
        r.pop('Account', None)

    # Convertir a DataFrame
    df = pd.DataFrame(registros)
    column_order = [
    'AccountId',
    'Account.CaseSafe_Id__c',
    'CaseNumber',
    'Account.Alt_Email__c',
    'Description',
    'Status',
    'Account.CreatedDate',
    'Closed_Date__c',
    'Case_Owner_read_only__c',
    'LeadName__c',
    'Organization__c',
    'Sub_Status_Original__c',
    'Sub_Status_Recovery__c',
    'Sub_Status__c',
    'Account.BillingCountry',
    'LeadCountry__c',
    'Type',
    'Subject',
    'Related_Country__c'
    ]
    df = df.reindex(columns=column_order)

    # Guardar como CSV
    df.to_excel('reporte_casos.xlsx', index=False)
    print("📄 Reporte guardado como 'reporte_casos.xlsx'.")

except SalesforceAuthenticationFailed as e:
    print(f"❌ Error de autenticación: {e}")
except SalesforceMalformedRequest as e:
    print(f"❌ Error en la consulta SOQL: {e.content}")
except Exception as e:
    print(f"❌ Error inesperado: {e}")
    
    


Conexion exitosa: openeducation.my.salesforce.com
Versión: 59.0
📄 Reporte guardado como 'reporte_casos.xlsx'.
